In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/machineHack/MPP_Dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('finish')



finish


In [ ]:
!pip install catboost

     |████████████████████████████████| 65.8MB 90kB/s 


In [ ]:
# Import Necessary Libraries
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
# Random State
seed = 42
np.random.seed(seed)

In [ ]:
train  = pd.read_csv('/content/MPP_Dataset/Train.csv')
test   = pd.read_csv('/content/MPP_Dataset/Test.csv')

In [ ]:
train  = train.drop_duplicates().reset_index(drop=True)

In [ ]:
X_train = train.drop(['popularity'], axis = 1)
y_train = train['popularity']

In [ ]:
#cols = ['Category_1','Category_2'] 
cols = ['Category_2']
X_scaled = pd.get_dummies(X_train, columns=cols)
test_scaled = pd.get_dummies(test , columns=cols)

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_scaled)
test_scaled = scaler.transform(test_scaled)

# Display
display(X_scaled.shape, test_scaled.shape)

(15285, 12)

(12140, 12)

In [ ]:
# Instantiate CatBOOST
model_cat = CatBoostClassifier(metric_period=400,
                          random_state=seed,
                          learning_rate=0.01,
                          loss_function = 'MultiClass',
                          border_count=1500)   # Catboost

# Instantiate ExtraTreeClassifer
model_etr=ExtraTreesClassifier(
    n_estimators = 6000, max_depth = None, n_jobs = -1, random_state = seed, verbose = 1,bootstrap=True)     # ExtraTreesClassifier

# Instantiate RandomForestClassifier
model_rf=RandomForestClassifier(
    n_estimators = 9000,  max_depth = None, n_jobs = -1, random_state = seed, verbose = 1)    # RandomForestClassifier


# Combine all models(estimators) and use 'soft voting' -- Voting Classifier
voting=VotingClassifier(estimators = [(
    'CatBoost', model_cat), ('ETR', model_etr), ('RF', model_rf)], voting = 'soft', weights = [8, 6, 8])



# Fit the Model
voting.fit(X_scaled, y_train)

# Make Predictions and convert Predictions to Dataframe
cm_pred = pd.DataFrame(voting.predict_proba(test_scaled))
display(cm_pred)

0:	learn: 1.5825669	total: 152ms	remaining: 2m 32s
400:	learn: 0.4635001	total: 29.7s	remaining: 44.3s
800:	learn: 0.4377283	total: 59.2s	remaining: 14.7s
999:	learn: 0.4295014	total: 1m 13s	remaining: 0us


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 1402 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 3402 tasks      | elapsed:   40.2s
[Parallel(n_jobs=-1)]: Done 6000 out of 6000 | elapsed:  1.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 888 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 1588 tasks      | elapsed:   54.3s
[Parallel(n_jobs=-1)]: Done 2488 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 3588 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 4888 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 6388 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 8088 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: 

,0,1,2,3,4
0,0.002403,0.064266,0.598503,0.261213,0.073615
1,0.000045,0.005574,0.031738,0.955256,0.007386
2,0.000065,0.004153,0.055776,0.925023,0.014984
3,0.000066,0.005355,0.046603,0.933864,0.014112
4,0.000074,0.001058,0.008534,0.986190,0.004144
...,...,...,...,...,...
12135,0.000246,0.029402,0.155123,0.784889,0.030340
12136,0.000049,0.000910,0.009918,0.986506,0.002616
12137,0.000048,0.002572,0.020809,0.973723,0.002849
12138,0.000058,0.003624,0.014762,0.978950,0.002607


In [ ]:
cm_pred.to_csv('vc-pred.csv',index=False)